In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


데이터셋 경로

In [2]:
default_path = '../../jiyoon/asset/default/mhcld_puf_2022.csv'
processed1_path = '../../jiyoon/asset/prepro2/결측치 존재 행 모두 제거.csv'
processed2_path = '../../jiyoon/asset/prepro2/결측치 20%이하인 칼럼만 제거.csv'
processed3_path = '../../jiyoon/asset/prepro2/20%이하인 결측치 삭제된 데이터.csv'
processed4_path = '../../jiyoon/asset/prepro2/지역별 차이 존재하는 칼럼 삭제.csv'
processed5_path = '../../jiyoon/asset/prepro2/칼럼제거 안한 단순 행 제거 원핫.csv'
processed6_path = '../../jiyoon/asset/prepro2/칼럼제거 한 원핫.csv'


In [3]:
default = pd.read_csv(default_path)
processed1 = pd.read_csv(processed1_path)
processed2 = pd.read_csv(processed2_path)
processed3 = pd.read_csv(processed3_path)
processed4 = pd.read_csv(processed4_path)
processed5 = pd.read_csv(processed5_path)
processed6 = pd.read_csv(processed6_path)


In [4]:
print(default.shape)
print(processed1.shape)
print(processed2.shape)
print(processed3.shape)
print(processed4.shape)
print(processed5.shape)
print(processed6.shape)

(6957919, 40)
(1066085, 40)
(105400, 40)
(2321773, 40)
(4636146, 35)
(1066085, 77)
(4636146, 69)


In [5]:
default.describe(include='all')

,YEAR,AGE,EDUC,ETHNIC,RACE,GENDER,SPHSERVICE,CMPSERVICE,OPISERVICE,RTCSERVICE,...,ODDFLG,PDDFLG,PERSONFLG,SCHIZOFLG,ALCSUBFLG,OTHERDISFLG,STATEFIP,DIVISION,REGION,CASEID
count,6957919.0,6.957919e+06,6.957919e+06,6.957919e+06,6.957919e+06,6.957919e+06,6.957919e+06,6.957919e+06,6.957919e+06,6.957919e+06,...,6.957919e+06,6.957919e+06,6.957919e+06,6.957919e+06,6.957919e+06,6.957919e+06,6.957919e+06,6.957919e+06,6.957919e+06,6.957919e+06
mean,2022.0,6.906807e+00,-2.893766e+00,1.930815e+00,2.941775e+00,1.509873e+00,1.983040e+00,1.029606e+00,1.956536e+00,1.989749e+00,...,2.131284e-02,1.593882e-02,2.298906e-02,1.065296e-01,5.158568e-02,1.265233e-01,2.766003e+01,5.462245e+00,2.740285e+00,2.022348e+10
std,0.0,4.054777e+00,6.287160e+00,4.495802e+00,4.548226e+00,7.047036e-01,1.291224e-01,1.694973e-01,2.038982e-01,1.007286e-01,...,1.444251e-01,1.252389e-01,1.498685e-01,3.085142e-01,2.211891e-01,3.324382e-01,1.597620e+01,2.517557e+00,1.069379e+00,2.008578e+06
min,2022.0,-9.000000e+00,-9.000000e+00,-9.000000e+00,-9.000000e+00,-9.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,2.022000e+10
25%,2022.0,3.000000e+00,-9.000000e+00,3.000000e+00,3.000000e+00,1.000000e+00,2.000000e+00,1.000000e+00,2.000000e+00,2.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.200000e+01,3.000000e+00,2.000000e+00,2.022174e+10
50%,2022.0,7.000000e+00,-9.000000e+00,4.000000e+00,5.000000e+00,2.000000e+00,2.000000e+00,1.000000e+00,2.000000e+00,2.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.700000e+01,5.000000e+00,3.000000e+00,2.022348e+10
75%,2022.0,1.000000e+01,4.000000e+00,4.000000e+00,5.000000e+00,2.000000e+00,2.000000e+00,1.000000e+00,2.000000e+00,2.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.200000e+01,8.000000e+00,4.000000e+00,2.022522e+10
max,2022.0,1.400000e+01,5.000000e+00,4.000000e+00,6.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.900000e+01,9.000000e+00,4.000000e+00,2.022696e+10


In [ ]:
processed1.describe(include='all')

,YEAR,AGE,GENDER,SPHSERVICE,CMPSERVICE,OPISERVICE,RTCSERVICE,IJSSERVICE,MH1,MH2,...,DETNLF_NotApplicable,DIVISION_1,DIVISION_2,DIVISION_3,DIVISION_4,DIVISION_5,DIVISION_6,DIVISION_7,DIVISION_8,DIVISION_9
count,4636146.0,4.636146e+06,4.636146e+06,4.636146e+06,4.636146e+06,4.636146e+06,4.636146e+06,4.636146e+06,4.636146e+06,4.636146e+06,...,4636146,4636146,4636146,4636146,4636146,4636146,4636146,4636146,4636146,4636146
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,2,2,2,2,2,2,2,2,2
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,False,False,False,False,False,False,False,False,False
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3694901,4553844,3908235,4193530,4145853,3843165,4294848,4187745,3995236,3969531
mean,2022.0,4.681176e-01,4.622738e-01,1.617119e-02,9.696720e-01,4.772887e-02,1.138532e-02,8.706154e-03,6.184109e+00,-5.776206e+01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.0,3.077598e-01,4.985748e-01,1.261336e-01,1.714882e-01,2.131920e-01,1.060929e-01,9.289972e-02,3.927211e+00,5.113751e+01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,2022.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,-9.900000e+01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,2022.0,1.538462e-01,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,-9.900000e+01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,2022.0,4.615385e-01,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e+00,-9.900000e+01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,2022.0,6.923077e-01,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.000000e+00,2.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
for col in default.columns:
    print(f"{col}: {default[col].unique()}")

YEAR: [2022]
AGE: [14  7  8  3  4  6  5 11  9  1 12 13  2 10 -9]
EDUC: [-9  3  4  2  5  1]
ETHNIC: [ 3  4  1  2 -9]
RACE: [ 3  4 -9  2  6  5  1]
GENDER: [ 2  1 -9]
SPHSERVICE: [2 1]
CMPSERVICE: [1 2]
OPISERVICE: [2 1]
RTCSERVICE: [2 1]
IJSSERVICE: [2 1]
MH1: [11  7 -9  6 13  5  1  2  3  8  4 10  9 12]
MH2: [-9 11  2  1  3 13  7  8 10  6  5  9  4 12]
MH3: [-9  2  7  1  4  8 11 10  9  3 13  5 12  6]
SUB: [-9  6  8  3 11  4  5 10  9 13 12  7  1  2]
MARSTAT: [ 1  2  4 -9  3]
SMISED: [-9  1  3  2]
SAP: [-9  1  2]
EMPLOY: [-9  4  5  1  2  3]
DETNLF: [-9  1  2  5  3  4]
VETERAN: [-9  2  1]
LIVARAG: [-9  2  3  1]
NUMMHS: [1 2 0 3]
TRAUSTREFLG: [0 1]
ANXIETYFLG: [0 1]
ADHDFLG: [0 1]
CONDUCTFLG: [0 1]
DELIRDEMFLG: [0 1]
BIPOLARFLG: [0 1]
DEPRESSFLG: [0 1]
ODDFLG: [0 1]
PDDFLG: [0 1]
PERSONFLG: [0 1]
SCHIZOFLG: [1 0]
ALCSUBFLG: [0 1]
OTHERDISFLG: [0 1]
STATEFIP: [ 1  2  4  5  6  8  9 10 11 12 13 15 16 17 18 19 20 21 22 24 25 26 27 28
 29 30 31 32 33 34 35 36 37 38 39 40 41 42 44 45 46 47 48 49 50